In [2]:
import os
import json
import torch
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import shutil
import torchvision
#import mediapipe as mp
import ultralytics
import yaml
import time
from tqdm import tqdm
from PIL import Image
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
data_root = "D:\\human_fall\\data"

train_root = f'{data_root}\\train\\images'
train_resize_images = f'{data_root}\\train\\train_resize_images'
labels_root = f'{data_root}\\train\\labels'

valid_root = f'{data_root}\\valid\\images'
valid_resize_images = f'{data_root}\\valid\\valid_resize_images'
valid_labels_root = f'{data_root}\\valid\\labels'


In [11]:

# Create directories if not exist
os.makedirs(train_resize_images, exist_ok=True)
os.makedirs(valid_resize_images, exist_ok=True)

# Helper function to resize and pad images
def letterbox_image(image, target_size=(640, 640)):
    # 원본 이미지 크기
    h, w = image.shape[:2]
    target_w, target_h = target_size

    # 비율 계산
    scale = min(target_w / w, target_h / h)

    # 새로운 이미지 크기
    new_w = int(w * scale)
    new_h = int(h * scale)

    # 이미지 리사이즈
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # 패딩 적용
    pad_w = (target_w - new_w) // 2
    pad_h = (target_h - new_h) // 2

    # 이미지를 타겟 크기로 채워서 새로운 이미지를 만듦
    padded_image = cv2.copyMakeBorder(resized_image, pad_h, target_h - new_h - pad_h, pad_w, target_w - new_w - pad_w,
                                      cv2.BORDER_CONSTANT, value=[128, 128, 128])

    return padded_image

# Process Non-Fall Data
for file in os.listdir(valid_root):
    file_path = os.path.join(valid_root, file)
    image = cv2.imread(file_path)
    resized_image = letterbox_image(image)
    output_path = os.path.join(valid_resize_images, file)
    cv2.imwrite(output_path, resized_image)

# Process Fall Data
#for file in os.listdir(fall_image_path):
    #file_path = os.path.join(fall_image_path, file)
    #image = cv2.imread(file_path)
    #resized_image = letterbox_image(image)
    #output_path = os.path.join(output_fall_image_path, file)
    #cv2.imwrite(output_path, resized_image)

In [ ]:

# Helper function to get all patient directories
def get_patient_dirs(path):
    patient_dirs = {}
    for item in os.listdir(path):
        patient_id = '_'.join(item.split('_')[:4])  # Get unique identifier
        if patient_id not in patient_dirs:
            patient_dirs[patient_id] = []
        patient_dirs[patient_id].append(os.path.join(path, item))
    return list(patient_dirs.values())

# Process Non-Fall Data
non_fall_patients_video = get_patient_dirs(non_fall_video_path)
non_fall_patients_image = get_patient_dirs(non_fall_image_path)

for patient_files in non_fall_patients_video:
    for file in patient_files:
        shutil.copy(file, output_nonfall_video_path)

for patient_files in non_fall_patients_image:
    for file in patient_files:
        shutil.copy(file, output_nonfall_image_path)

# Process Fall Data
fall_patients_video = []
fall_patients_image = []
for path in fall_video_paths:
    fall_patients_video.extend(get_patient_dirs(path))
for path in fall_image_paths:
    fall_patients_image.extend(get_patient_dirs(path))

selected_patients = random.sample(fall_patients_video, len(fall_patients_video) // 3)

for patient_files in selected_patients:
    for file in patient_files:
        shutil.copy(file, output_fall_video_path)

# Ensure image selection matches selected patients
selected_patient_ids = {'_'.join(files[0].split('_')[:4]) for files in selected_patients}

for patient_files in fall_patients_image:
    patient_id = '_'.join(patient_files[0].split('_')[:4])
    if patient_id in selected_patient_ids:
        for file in patient_files:
            shutil.copy(file, output_fall_image_path)

In [14]:
# json 파일에 있는 bbox 정보를 yolo 형식으로
# class_name x_center y_center w h

def cvt_bbox_yolo(img_w, img_h, bbox):
    x, y, w, h = map(float, bbox.split(', '))
    
    x_center = (x + w/2) / img_w
    y_center = (y + h/2) / img_h
    
    width = w / img_w
    height = h / img_h
    
    return x_center, y_center, width, height

def cvt_json_yolo(json_data):
    # 비낙상 : N, 낙상 : Y
    classes = {'N' : 0, 'Y' : 1}
    img_w, img_h = map(int, json_data['metadata']['scene_res'].split(' x '))
    bbox = json_data['bboxdata']['bbox_location']
    x_center, y_center, width, height = cvt_bbox_yolo(img_w, img_h, bbox)  
    
    img_path = json_data['img_path']['img_path']
    class_name = img_path.split('/')[1]
    
    try : 
        class_id = classes[class_name]
    except KeyError : 
        print('클래스 정보 잘못')
        return None
    
    yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"    
    return yolo_format

def json_files_out(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            json_path = os.path.join(input_dir, filename)
            
            # json read
            try : 
                with open(json_path, 'r', encoding = 'utf-8') as f:
                    json_data = json.load(f)
            
                # YOLO 형식으로 convert
                yolo_format = cvt_json_yolo(json_data)
                if yolo_format is None : 
                    print(f'건너뜀 : {filename}')
                    continue
            
                # 파일로 저장
                output_filename = json_data['metadata']['file_name'].replace('.JPG', '.txt')
                output_path = os.path.join(output_dir, output_filename)
            
                with open(output_path, 'w') as f:
                    f.write(yolo_format)
            
                print('완료')
            except Exception as e : 
                print(f'에러 {filename} : {str(e)}')

input_dir = "D:\\human_fall\\data\\json"
output_dir = labels_root

json_files_out(input_dir, output_dir)

완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완

In [16]:
def normalize_coordinates(label_path, image_path):
    # 이미지 크기 가져오기
    with Image.open(image_path) as img:
        img_width, img_height = img.size
    
    # 라벨 파일 읽기
    with open(label_path, 'r') as f:
        lines = f.readlines()
    
    normalized_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:  # class x y width height 형식 확인
            class_id = parts[0]
            x = float(parts[1])
            y = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            
            # 좌표 정규화
            x_normalized = x / img_width
            y_normalized = y / img_height
            width_normalized = width / img_width
            height_normalized = height / img_height
            
            # 정규화된 좌표로 새 라인 생성
            new_line = f"{class_id} {x_normalized:.6f} {y_normalized:.6f} {width_normalized:.6f} {height_normalized:.6f}\n"
            normalized_lines.append(new_line)
    
    # 정규화된 좌표로 파일 다시 쓰기
    with open(label_path, 'w') as f:
        f.writelines(normalized_lines)

image_folder = valid_root
label_folder = valid_labels_root

for filename in os.listdir(label_folder):
    if filename.endswith('.txt'):
        label_path = os.path.join(label_folder, filename)
        image_path = os.path.join(image_folder, filename.replace('.txt', '.jpg'))
        
        if os.path.exists(image_path):
            normalize_coordinates(label_path, image_path)
        else:
            print(f"Image not found for label: {filename}")

In [17]:
!cd 'D:\\human_fall\\data'

���� �̸�, ���͸� �̸� �Ǵ� ���� ���̺� ������ �߸��Ǿ����ϴ�.


In [5]:
class_names = {0 : '비낙상', 1 : '낙상'}
num_classes = len(class_names)

yaml_info = {
    'names': class_names,
    'nc': num_classes,
    'train': train_root,
    'val': valid_root
}

with open('yaml_info.yaml', 'w') as f : 
    yaml.dump(yaml_info, f)
print(f'이 경로에 yaml파일 생성 : {data_root}')

이 경로에 yaml파일 생성 : D:\human_fall\data


In [6]:
start_time = time.time()

model = YOLO('yolov8s.pt')
result = model.train(data = 'yaml_info.yaml', epochs = 50, batch = 16, imgsz =640, device = device, workers = 20, amp = False, patience = 30, name = 'human_fall_s')

end_time = time.time()
execution_time = end_time - start_time
print(f"실행 시간: {execution_time:.4f} 초")

New https://pypi.org/project/ultralytics/8.3.14 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=yaml_info.yaml, epochs=50, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=20, project=None, name=human_fall_s13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning D:\human_fall\data\train\labels... 181199 images, 80 backgrounds, 0 corrupt: 100%|██████████| 181279/181279 [03:18<00:00, 912.14it/s] 


train: New cache created: D:\human_fall\data\train\labels.cache


val: Scanning D:\human_fall\data\valid\labels... 22720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22720/22720 [02:41<00:00, 140.28it/s]


val: New cache created: D:\human_fall\data\valid\labels.cache
Plotting labels to runs\detect\human_fall_s13\labels.jpg... 


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\utils\plotting.py:921: UserWarning: Glyph 48708 (\N{HANGUL SYLLABLE BI}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\utils\plotting.py:921: UserWarning: Glyph 45209 (\N{HANGUL SYLLABLE NAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\utils\plotting.py:921: UserWarning: Glyph 49345 (\N{HANGUL SYLLABLE SANG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\utils\plotting.py:921: UserWarning: Glyph 48708 (\N{HANGUL SYLLABLE BI}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\utils\plotting.py:921: UserWa

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 20 dataloader workers
Logging results to runs\detect\human_fall_s13
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      7.02G          0      101.3          0          0        640:   4%|▎         | 406/11330 [01:54<51:19,  3.55it/s]  


KeyboardInterrupt: 